# 工厂规划 I

## 目标和前提条件

想学习如何制定能最大化利润的最优生产计划吗？在这个示例中，我们将教您如何解决这个经典的生产规划问题。

关于这类模型的更多信息可以在 H. P. Williams 所著《数学规划中的建模》第五版的示例 #3（第 255-256 页和第 300-302 页）中找到。

这个建模示例属于中级水平，我们假设您了解 Python 并熟悉 Gurobi Python API。此外，您还应该具备一些数学优化模型构建的知识。

**下载代码库** <br />
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

---
## 问题描述

一家工厂生产七种产品（产品1至产品7），使用多台机器，包括：

- 四台磨床
- 两台立式钻床
- 三台卧式钻床
- 一台镗床
- 一台刨床

每种产品都有明确的单位销售利润贡献（定义为单位销售价格减去原材料成本）。此外，每种产品的制造都需要在每台机器上花费一定的时间（以小时为单位）。利润贡献和制造时间如下表所示。横线表示该产品的制造过程不需要使用该机器。

| <i></i> | 产品1 | 产品2 | 产品3 | 产品4 | 产品5 | 产品6 | 产品7 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 利润 | 10 | 6 | 8 | 4 | 11 | 9 | 3 |
| 磨床 | 0.5 | 0.7 | - | - | 0.3 | 0.2 | 0.5 |
| 立式钻床 | 0.1 | 0.2 | - | 0.3 | - | 0.6 | - |
| 卧式钻床 | 0.2 | - | 0.8 | - | - | - | 0.6 |
| 镗床 | 0.05 | 0.03 | - | 0.07 | 0.1 | - | 0.08 |
| 刨床 | - | - | 0.01 | - | 0.05 | - | 0.05 |

在此模型涵盖的六个月中，每个月都有一台或多台机器需要进行维护，因此在该月不能用于生产。维护计划如下：

| 月份 | 机器 |
| --- | --- |
| 一月 | 一台磨床 |
| 二月 | 两台卧式钻床 |
| 三月 | 一台镗床 |
| 四月 | 一台立式钻床 |
| 五月 | 一台磨床和一台立式钻床 |
| 六月 | 一台卧式钻床 |

每个月每种产品的销售量都有限制。这些限制如下所示：

| 月份 | 产品1 | 产品2 | 产品3 | 产品4 | 产品5 | 产品6 | 产品7 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 一月 | 500 | 1000 | 300 | 300 | 800 | 200 | 100 |
| 二月 | 600 | 500 | 200 | 0 | 400 | 300 | 150 |
| 三月 | 300 | 600 | 0 | 0 | 500 | 400 | 100 |
| 四月 | 200 | 300 | 400 | 500 | 200 | 0 | 100 |
| 五月 | 0 | 100 | 500 | 100 | 1000 | 300 | 0 |
| 六月 | 500 | 500 | 100 | 300 | 1100 | 500 | 60 |

每种产品最多可以存储100个单位的库存，存储成本为每单位每月0.50美元。在一月初，没有产品库存。然而，到六月底，每种产品的库存应该有50个单位。

工厂每周工作六天，每天两班，每班八小时。可以假设每个月有24个工作日。另外，就本模型而言，不需要考虑生产排序问题。

生产计划应该是什么样的？此外，是否可以建议任何价格上调，并确定购置新机器的价值？

这个问题是基于为康沃尔工程公司 Holman Brothers 建立的一个更大模型。

---
## 模型公式

### 集合和索引

$t \in \text{Months}=\{\text{一月},\text{二月},\text{三月},\text{四月},\text{五月},\text{六月}\}$：月份集合

$p \in \text{Products}=\{1,2,\dots,7\}$：产品集合

$m \in \text{Machines}=\{\text{磨床},\text{立式钻床},\text{卧式钻床},\text{镗床},\text{刨床}\}$：机器集合

### 参数

$\text{hours_per_month} \in \mathbb{R}^+$：每月任何机器的可用时间（小时/月）。这是工作日数（24天）乘以每天班次数（2）再乘以每班时长（8小时）的结果。

$\text{max_inventory} \in \mathbb{N}$：任何给定月份中单个产品类型可存储的最大数量。

$\text{holding_cost} \in \mathbb{R}^+$：每月保持任何产品类型一个单位的库存成本（美元/单位/月）。

$\text{store_target} \in \mathbb{N}$：规划期末需要保持的每种产品类型的库存单位数。

$\text{profit}_p \in \mathbb{R}^+$：产品$p$的利润（美元/单位）。

$\text{installed}_m \in \mathbb{N}$：工厂安装的$m$类型机器数量。

$\text{down}_{t,m} \in \mathbb{N}$：$t$月份计划维护的$m$类型机器数量。

$\text{time_req}_{m,p} \in \mathbb{R}^+$：在机器$m$上制造一个单位的产品$p$所需的时间（小时/单位）。

$\text{max_sales}_{t,p} \in \mathbb{N}$：$t$月份产品$p$的最大销售单位数。


### 决策变量

$\text{make}_{t,p} \in \mathbb{R}^+$：$t$月份生产产品$p$的单位数。

$\text{store}_{t,p} \in [0, \text{max_inventory}] \subset \mathbb{R}^+$：$t$月份存储产品$p$的单位数。

$\text{sell}_{t,p} \in [0, \text{max_sales}_{t,p}] \subset \mathbb{R}^+$：$t$月份销售产品$p$的单位数。

**假设：** 我们可以生产小数单位。

### 目标函数

- **利润：** 最大化规划期内的总利润（美元）。

\begin{equation}
\text{最大化} \quad Z = \sum_{t \in \text{Months}}\sum_{p \in \text{Products}}
(\text{profit}_p*\text{make}_{t,p} - \text{holding_cost}*\text{store}_{t,p})
\tag{0}
\end{equation}

### 约束条件

- **初始平衡：** 对于每个产品$p$，生产的单位数应等于销售的单位数加上存储的单位数（以产品单位计）。

\begin{equation}
\text{make}_{\text{一月},p} = \text{sell}_{\text{一月},p} + \text{store}_{\text{一月},p} \quad \forall p \in \text{Products}
\tag{1}
\end{equation}

- **平衡：** 对于每个产品$p$，$t$月份生产的单位数和之前存储的单位数应等于该月销售和存储的单位数（以产品单位计）。

\begin{equation}
\text{store}_{t-1,p} + \text{make}_{t,p} = \text{sell}_{t,p} + \text{store}_{t,p} \quad \forall (t,p) \in \text{Months} \setminus \{\text{一月}\} \times \text{Products}
\tag{2}
\end{equation}

- **库存目标：** 规划期末每个产品$p$的库存单位数应达到目标（以产品单位计）。

\begin{equation}
\text{store}_{\text{六月},p} = \text{store_target} \quad \forall p \in \text{Products}
\tag{3}
\end{equation}

- **机器产能：** 在机器类型$m$上制造任何产品所用的总时间不能超过其每月产能（小时）。

\begin{equation}
\sum_{p \in \text{Products}}\text{time_req}_{m,p}*\text{make}_{t,p} \leq \text{hours_per_month}*(\text{installed}_m - \text{down}_{t,m}) \quad \forall (t,m) \in \text{Months} \times \text{Machines}
\tag{4}
\end{equation}

---
## Python实现

我们导入Gurobi Python模块和其他Python库。

In [ ]:
# %pip install gurobipy

In [1]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.11 & Gurobi 11.0

### 输入数据
我们定义模型的所有输入数据。

In [ ]:
# 参数

products = ["Prod1", "Prod2", "Prod3", "Prod4", "Prod5", "Prod6", "Prod7"]
machines = ["grinder", "vertDrill", "horiDrill", "borer", "planer"]
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun"]

profit = {"Prod1":10, "Prod2":6, "Prod3":8, "Prod4":4, "Prod5":11, "Prod6":9, "Prod7":3}

time_req = {
    "grinder": {    "Prod1": 0.5, "Prod2": 0.7, "Prod5": 0.3,
                    "Prod6": 0.2, "Prod7": 0.5 },
    "vertDrill": {  "Prod1": 0.1, "Prod2": 0.2, "Prod4": 0.3,
                    "Prod6": 0.6 },
    "horiDrill": {  "Prod1": 0.2, "Prod3": 0.8, "Prod7": 0.6 },
    "borer": {      "Prod1": 0.05,"Prod2": 0.03,"Prod4": 0.07,
                    "Prod5": 0.1, "Prod7": 0.08 },
    "planer": {     "Prod3": 0.01,"Prod5": 0.05,"Prod7": 0.05 }
}


# 停机的机器数量
down = {("Jan","grinder"): 1, ("Feb", "horiDrill"): 2, ("Mar", "borer"): 1,
        ("Apr", "vertDrill"): 1, ("May", "grinder"): 1, ("May", "vertDrill"): 1,
        ("Jun", "planer"): 1, ("Jun", "horiDrill"): 1}

# 每种机器的可用数量
installed = {"grinder":4, "vertDrill":2, "horiDrill":3, "borer":1, "planer":1} 

# 销售的市场限制
max_sales = {
    ("Jan", "Prod1") : 500,
    ("Jan", "Prod2") : 1000,
    ("Jan", "Prod3") : 300,
    ("Jan", "Prod4") : 300,
    ("Jan", "Prod5") : 800,
    ("Jan", "Prod6") : 200,
    ("Jan", "Prod7") : 100,
    ("Feb", "Prod1") : 600,
    ("Feb", "Prod2") : 500,
    ("Feb", "Prod3") : 200,
    ("Feb", "Prod4") : 0,
    ("Feb", "Prod5") : 400,
    ("Feb", "Prod6") : 300,
    ("Feb", "Prod7") : 150,
    ("Mar", "Prod1") : 300,
    ("Mar", "Prod2") : 600,
    ("Mar", "Prod3") : 0,
    ("Mar", "Prod4") : 0,
    ("Mar", "Prod5") : 500,
    ("Mar", "Prod6") : 400,
    ("Mar", "Prod7") : 100,
    ("Apr", "Prod1") : 200,
    ("Apr", "Prod2") : 300,
    ("Apr", "Prod3") : 400,
    ("Apr", "Prod4") : 500,
    ("Apr", "Prod5") : 200,
    ("Apr", "Prod6") : 0,
    ("Apr", "Prod7") : 100,
    ("May", "Prod1") : 0,
    ("May", "Prod2") : 100,
    ("May", "Prod3") : 500,
    ("May", "Prod4") : 100,
    ("May", "Prod5") : 1000,
    ("May", "Prod6") : 300,
    ("May", "Prod7") : 0,
    ("Jun", "Prod1") : 500,
    ("Jun", "Prod2") : 500,
    ("Jun", "Prod3") : 100,
    ("Jun", "Prod4") : 300,
    ("Jun", "Prod5") : 1100,
    ("Jun", "Prod6") : 500,
    ("Jun", "Prod7") : 60,
}

holding_cost = 0.5
max_inventory = 100
store_target = 50
hours_per_month = 2*8*24

## 模型部署
我们创建一个模型和变量。对于每种产品（七种产品）和每个时间段（月份），我们将创建变量来表示生产、存储和销售的产品数量。在每个月份中，每种产品的销售数量都有上限。这是由于市场限制。

In [ ]:
factory = gp.Model('Factory Planning I')

make = factory.addVars(months, products, name="Make") # 生产数量
store = factory.addVars(months, products, ub=max_inventory, name="Store") # 存储数量
sell = factory.addVars(months, products, ub=max_sales, name="Sell") # 销售数量

Using license file c:\gurobi\gurobi.lic


接下来，我们插入约束条件。平衡约束确保上一个月的库存量加上当月生产的数量等于当月销售和持有的数量，这适用于每种产品。这确保了所有产品都在某个月份生产。初始库存为空。

In [ ]:
#1. 初始平衡
Balance0 = factory.addConstrs((make[months[0], product] == sell[months[0], product] 
                  + store[months[0], product] for product in products), name="Initial_Balance")
    
#2. 平衡
Balance = factory.addConstrs((store[months[months.index(month) -1], product] + 
                make[month, product] == sell[month, product] + store[month, product] 
                for product in products for month in months 
                if month != months[0]), name="Balance")

库存目标约束强制要求在最后一个月月末，库存中包含每种产品指定数量。

In [ ]:
#3. 库存目标
TargetInv = factory.addConstrs((store[months[-1], product] == store_target for product in products),  name="End_Balance")

产能约束确保在每个月份，所有产品在某类机器上所需的时间小于或等于该类机器在该月的可用小时数乘以该期间可用的机器数量。每种产品在不同机器上都需要一定的机器小时数。由于维护原因，每台机器在一个或多个月份都会停机，因此每月可用的机器数量和类型各不相同。每种机器类型可以有多台机器。

In [ ]:
#4. 机器产能

MachineCap = factory.addConstrs((gp.quicksum(time_req[machine][product] * make[month, product]
                             for product in time_req[machine])
                    <= hours_per_month * (installed[machine] - down.get((month, machine), 0))
                    for machine in machines for month in months),
                   name = "Capacity")

目标是最大化公司的利润，包括每种产品的利润减去存储未售产品的成本。可以表述为：

In [ ]:
#0. 目标函数
obj = gp.quicksum(profit[product] * sell[month, product] -  holding_cost * store[month, product]  
               for month in months for product in products)

factory.setObjective(obj, GRB.MAXIMIZE)

接下来，我们开始优化，Gurobi将找到最优解。

In [8]:
factory.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 79 rows, 126 columns and 288 nonzeros
Model fingerprint: 0xead11e9d
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [5e-01, 1e+01]
  Bounds range     [6e+01, 1e+03]
  RHS range        [5e+01, 2e+03]
Presolve removed 74 rows and 110 columns
Presolve time: 0.01s
Presolved: 5 rows, 16 columns, 21 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2466500e+05   3.640000e+02   0.000000e+00      0s
Extra simplex iterations after uncrush: 2
       4    9.3715179e+04   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds
Optimal objective  9.371517857e+04


---
## 分析

优化模型的结果显示，我们可以实现的最大利润是$\$93,715.18$。
让我们看看达到这个最优结果的解决方案。

### 生产计划

这个计划确定了在规划期内每个时期生产每种产品的数量。例如，在二月我们生产700个产品1。

In [9]:
rows = months.copy()
columns = products.copy()
make_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, product in make.keys():
    if (abs(make[month, product].x) > 1e-6):
        make_plan.loc[month, product] = np.round(make[month, product].x, 1)
make_plan

,Prod1,Prod2,Prod3,Prod4,Prod5,Prod6,Prod7
Jan,500.0,888.6,382.5,300.0,800.0,200.0,0.0
Feb,700.0,600.0,117.5,0.0,500.0,300.0,250.0
Mar,0.0,0.0,0.0,0.0,0.0,400.0,0.0
Apr,200.0,300.0,400.0,500.0,200.0,0.0,100.0
May,0.0,100.0,600.0,100.0,1100.0,300.0,100.0
Jun,550.0,550.0,0.0,350.0,0.0,550.0,0.0


### 销售计划

这个计划定义了在规划期内每个时期销售每种产品的数量。例如，在二月我们销售600个产品1。

In [10]:
rows = months.copy()
columns = products.copy()
sell_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, product in sell.keys():
    if (abs(sell[month, product].x) > 1e-6):
        sell_plan.loc[month, product] = np.round(sell[month, product].x, 1)
sell_plan

,Prod1,Prod2,Prod3,Prod4,Prod5,Prod6,Prod7
Jan,500.0,888.6,300.0,300.0,800.0,200.0,0.0
Feb,600.0,500.0,200.0,0.0,400.0,300.0,150.0
Mar,100.0,100.0,0.0,0.0,100.0,400.0,100.0
Apr,200.0,300.0,400.0,500.0,200.0,0.0,100.0
May,0.0,100.0,500.0,100.0,1000.0,300.0,0.0
Jun,500.0,500.0,50.0,300.0,50.0,500.0,50.0


### 库存计划

这个计划反映了在规划期内每个时期结束时库存中的产品数量。例如，在二月底我们有100个产品1的库存。

In [11]:
rows = months.copy()
columns = products.copy()
store_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for month, product in store.keys():
    if (abs(store[month, product].x) > 1e-6):
        store_plan.loc[month, product] = np.round(store[month, product].x, 1)
store_plan

,Prod1,Prod2,Prod3,Prod4,Prod5,Prod6,Prod7
Jan,0.0,0.0,82.5,0.0,0.0,0.0,0.0
Feb,100.0,100.0,0.0,0.0,100.0,0.0,100.0
Mar,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Apr,0.0,0.0,0.0,0.0,0.0,0.0,0.0
May,0.0,0.0,100.0,0.0,100.0,0.0,100.0
Jun,50.0,50.0,50.0,50.0,50.0,50.0,50.0


**注意：** 如果您想将解决方案写入文件，而不是打印到终端，可以使用model.write()命令。示例实现为：

`factory.write("factory-planning-1-output.sol")`

---
## 参考文献

H. Paul Williams，《数学规划中的模型构建》，第五版。

版权所有 &copy; 2020 Gurobi Optimization, LLC